In [1]:
import numpy as np
import pandas as pd
import os
os.environ['TF_ENABLE_ONEDNN_OPTS']='0'
import tensorflow as tf
from tensorflow import keras

In [2]:
ALL_DATA_DIR = '/media/windows/'
training = ALL_DATA_DIR + 'training/'
validation = ALL_DATA_DIR + 'validation/'
scaled = ALL_DATA_DIR + 'scaled/'
scaled_wo_rotate = ALL_DATA_DIR + "scaled_wo_rotate/"

scaled_wo_rotate_training = scaled_wo_rotate + "training/"
scaled_wo_rotate_validation = scaled_wo_rotate + "validation/"

datFile = training + "shape_predictor_68_face_landmarks.dat"

In [3]:
batch_size = 32
img_height = 224
img_width = 224

In [4]:
validation_s = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_validation, labels = 'inferred',
            image_size=(img_height,img_width), batch_size=batch_size, seed=42).prefetch(1)

Found 50947 files belonging to 7 classes.


2022-09-07 02:27:50.112907: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-07 02:27:50.121770: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-07 02:27:50.124387: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-07 02:27:50.128999: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

In [5]:
training_s = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_training, labels = 'inferred',
            image_size=(img_height,img_width), batch_size=batch_size, seed=42).prefetch(1)

Found 399839 files belonging to 7 classes.


In [6]:
base_model = keras.applications.EfficientNetB4(input_shape=(224,224,3), include_top = False, weights=None)

In [7]:
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(7, activation='softmax')(avg)
model = keras.Model(inputs=base_model.input, outputs=output)

In [8]:
es = keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)

In [9]:
opt = keras.optimizers.Adam(learning_rate=1e-4)

In [10]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
history = model.fit(training_s, validation_data=validation_s, epochs=10, callbacks=[es])

Epoch 1/10


2022-09-07 02:30:08.341986: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8400


12495/12495 [==============================] - 9540s 762ms/step - loss: 1.4807 - accuracy: 0.4088 - val_loss: 1.0172 - val_accuracy: 0.6287
Epoch 2/10
12495/12495 [==============================] - 9521s 762ms/step - loss: 0.8980 - accuracy: 0.6647 - val_loss: 0.7890 - val_accuracy: 0.7094
Epoch 3/10
12495/12495 [==============================] - 9518s 762ms/step - loss: 0.7404 - accuracy: 0.7263 - val_loss: 0.7176 - val_accuracy: 0.7442
Epoch 4/10
12495/12495 [==============================] - 9510s 761ms/step - loss: 0.6267 - accuracy: 0.7704 - val_loss: 0.6882 - val_accuracy: 0.7630
Epoch 5/10
12495/12495 [==============================] - 9516s 762ms/step - loss: 0.5339 - accuracy: 0.8051 - val_loss: 0.6874 - val_accuracy: 0.7712
Epoch 6/10
12495/12495 [==============================] - 9560s 765ms/step - loss: 0.4604 - accuracy: 0.8319 - val_loss: 0.7031 - val_accuracy: 0.7737
Epoch 7/10
12495/12495 [==============================] - 9555s 765ms/step - loss: 0.3994 - accuracy: 0.8

In [11]:
model.save('0907_EffiB4_no_weight.h5')

In [12]:
model.evaluate(validation_s)

1593/1593 [==============================] - 234s 147ms/step - loss: 0.6874 - accuracy: 0.7712


[0.6873790621757507, 0.7711935639381409]

In [13]:
model.evaluate(training_s)

12495/12495 [==============================] - 1833s 147ms/step - loss: 0.3981 - accuracy: 0.8570


[0.39809003472328186, 0.8570424318313599]